In [51]:
! pip -q install langchain-community langchain-core langgraph langserve langchain-google-genai wikipedia

In [74]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from google.colab import userdata
import os

from langchain.agents import tool, AgentExecutor
from langchain.agents import AgentType, initialize_agent, load_tools, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub

In [53]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [54]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [55]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create Custom Tool

In [56]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [57]:
get_word_length.invoke("foo")

3

In [58]:
@tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers."""
    return a * b

In [59]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiplies two numbers.
{'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}


In [60]:
multiply.invoke({"a":2,"b":3})

6.0

# First Version Of Agents

In [61]:
tool = load_tools(["wikipedia"],llm=llm)

In [62]:
agent = initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [63]:
agent.invoke("What is the capital of France?")



> Entering new AgentExecutor chain...
Thought: I need to find the capital city of France.  Wikipedia is a good resource for this.
Action: wikipedia
Action Input: Paris

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Paris
Summary: Paris (French pronunciation: [paʁi] ) is the capital and largest city of France. With an estimated population of 2,102,650 residents in January 2023 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-most populous city in the European Union, the ninth-most populous city in Europe and the 30th most densely populated city in the world in 2022. Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, culture, fashion, and gastronomy. Because of its leading role in the arts and sciences and its early adaptation of extensive street lighting, it became known as the City of Light in the 19th century.
The City of Paris is the centre of the Île-de-France region, or Paris Region, with an official estimated population of 12,271,794 inhabitants in January 2023, or about 19% of the population of France. The Paris Region had a nominal GDP of €765 billion (US$1.064 trillion when adjusted for PPP) in 2021, 

{'input': 'What is the capital of France?', 'output': 'Paris'}

# Second Version

In [64]:
search = TavilySearchResults()

In [65]:
search.invoke("What is the capital of France?")

[{'url': 'https://simple.m.wikipedia.org/wiki/Capital_of_France',
  'content': 'Learn about the history and current status of the capital of France, which is Paris. Find out which other cities have been the national capital in different periods of time.'},
 {'url': 'https://simple.m.wikipedia.org/wiki/Paris',
  'content': 'Paris Paris (nicknamed the "City of light") is the capital city of France, and the largest city in France. Paris Paris Paris Paris Paris (Earth) | Department | Paris | When Hugh Capet became king of France in 987, he again made Paris his capital. For a long time, the kings only controlled Paris and the surrounding area, as much of the rest of France was in the hands of barons or English. ↑ "Dossier complet − Aire urbaine de Paris (001) | Insee". ↑ "Paris". ↑ "5 Fun Facts about Paris". ↑ "Climatological Information for Paris, France". ↑ "Normes et records 1961–1990: Paris-Montsouris (75) – altitude 75m" (in French). ↑ "Paris, France – Climate data". ↑ "Paris (07150) -

In [66]:
tools = [search]

In [70]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [73]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [72]:
agent = create_tool_calling_agent(llm,tools,prompt)

In [75]:
agent_executer = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [80]:
agent_executer.invoke({"input":"tell me about yourself"})



> Entering new AgentExecutor chain...
I'm Gemini, a large language model created by Google.  I can process information and respond to your questions in a helpful and informative way.  I'm still under development, but I'm learning new things all the time.


> Finished chain.


{'input': 'tell me about yourself',
 'output': "I'm Gemini, a large language model created by Google.  I can process information and respond to your questions in a helpful and informative way.  I'm still under development, but I'm learning new things all the time.\n"}